<a href="https://colab.research.google.com/github/jeanmichael1993/Projeto_lotofacil/blob/master/redesneuraisLotofacil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def carregar_dados() -> str:
  try:
    dados = ''
    with open('Lotofacil2.txt', 'r') as arquivo:
      dados = arquivo.readlines()
    return dados
  except TypeError as error:
    print(error)


def tratar_dados(dados: str) -> str:
  novos: str = []
  for i in dados:
    novos.append(i.rstrip().replace("\n", ' ').split(" "))
  for i in novos:
    print(i)
  return novos


if __name__ == "__main__":
  dados: str = carregar_dados()
  tratados: str = tratar_dados(dados)


In [ ]:
import numpy as np
def criaConjuntos(dados,W = 15):
    amostras = []
    alvo = []
    N = len(dados)
    for px in range(N-W):
        X = np.zeros((5,5,W),dtype=int)
        Y = np.zeros((5,5),dtype=int)
        for pw in range(W):
            for nro in dados[px+pw]:
                li = (int(nro)-1) // 5
                co = (int(nro)-1) % 5
                X[li,co,pw] = 1

        for nro in dados[px+W]:
            li = (int(nro)-1) // 5
            co = (int(nro)-1) % 5
            Y[li,co] = 1

        amostras.append(X)
        alvo.append(Y)

    return np.array(amostras),np.array(alvo)
amostras,alvo = criaConjuntos(tratados)
print("Amostras processadas")
print(amostras)

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
def converteSaidaBinarioN15(x):
    x = np.array(x)
    # Encontrar os índices dos 15 maiores valores
    indices_melhores = np.argpartition(x.flatten(), -15)[-15:]
    # Inicializar a matriz resultante com zeros
    matriz_resultante = np.zeros_like(x)
    # Definir os 15 maiores valores para 1
    matriz_resultante.flat[indices_melhores] = 1
    # Imprimir a matriz resultante
    matriz_resultante = keras.backend.cast(matriz_resultante, dtype=tf.int32) #will convert bool to 0 and 1
    return matriz_resultante
def metricFunction(y_true, y_pred, imprime = False):
    y_pred = converteSaidaBinarioN15(y_pred)
    y_pred = tf.squeeze(y_pred)
    quantidade_iguais = np.sum((y_true == 1) & (y_pred == 1))
    return quantidade_iguais / 15
p0 = keras.layers.Input((5, 5, 15))
p1 = keras.layers.Conv2D(8, (3, 3), padding="same", strides=1, activation="relu")(p0)
p2 = keras.layers.Conv2D(8, (3, 3), padding="same", strides=1, activation="relu")(p1)
p3 = keras.layers.Conv2D(16, (3, 3), padding="same", strides=1, activation="relu")(p2)
p4 = keras.layers.Conv2D(16, (3, 3), padding="same", strides=1, activation="relu")(p3)
p5 = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(p4)
model = keras.models.Model(p0, p5)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[metricFunction])
model.summary()

print("Modelo criado!")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 5, 5, 15)]        0         
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 8)           1088      
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 5, 8)           584       
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 16)          1168      
                                                                 
 conv2d_3 (Conv2D)           (None, 5, 5, 16)          2320      
                                                                 
 conv2d_4 (Conv2D)           (None, 5, 5, 1)           17        
                                                                 
Total params: 5177 (20.22 KB)
Trainable params: 5177 (20.22 K

In [ ]:
# prompt: rodar o modelo criado com as amostras

model.fit(amostras, alvo, epochs=10, batch_size=100)


ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, <class 'keras.src.engine.functional.Functional'>